从零开始构建Llama 3模型完整架构并使用自定义数据集进行训练和推理的分步指南。

下面的步骤参考博客:[Build Your Own Llama 3 Architecture from Scratch Using PyTorch](https://pub.towardsai.net/build-your-own-llama-3-architecture-from-scratch-using-pytorch-2ce1ecaa901c)
The related github: https://github.com/tamangmilan/llama3  



![llama3 Arch](images/llama3_arch.webp)  

【作者】：Llama 3架构展示了训练和推理流程。我想象出了这个图，因为官方的Llama 3论文中并没有提供。到本文结束时，我相信你应该能够画出比这更好的架构。

通过阅读这篇文章，你将实现以下目标：
1. 深入理解Llama 3模型每个组件的内部工作原理。
2. 编写代码构建Llama 3的每个组件，并将它们组装成一个完全功能的Llama 3模型。
3. 编写代码使用新的自定义数据集训练你的模型。
4. 编写代码进行推理，使你的Llama 3模型能够根据输入提示生成新文本。

先决条件
- 需要具备基本的Python和Pytorch知识。
- 对Transformer概念（如自注意力机制）的基本理解以及对深度神经网络的了解会有所帮助，但并非强制要求。

现在我们知道了想要实现的目标，让我们一步一步开始构建。

# 步骤一：输入块 
如上图所示的Llama 3架构图中，输入块包含三个组件：文本/提示、分词器和嵌入。

输入块内部的组件是如何工作的呢？有一句流行的话说“一图胜千言”，让我们看看下面的流程图来理解输入块内部的工作流程。

![Input Block](images/input_block.webp)

[作者提供的图片]：输入显示提示、分词器和嵌入流程的块流程图。

- 首先，单个或一批文本/提示将被输入到模型中。例如：上述流程图中的“Hello World”。
- 由于模型无法处理文本，因此输入到模型的内容必须始终采用数字格式。分词器有助于将这些文本/提示转换为标记ID（这是词汇表中标记的索引号表示）。我们将使用广为人知的《小小莎士比亚》数据集来构建词汇表，并训练我们的模型。
- Llama 3模型中使用的分词器是TikToken，这是一种子词分词器。然而，在构建我们的模型时，我们将使用字符级分词器。主要原因是我们应该了解如何自行构建词汇表和分词器，包括编码和解码功能。这样一来，我们就能了解底层的所有工作原理，并且可以完全掌控代码。
- 最后，每个标记ID将被转换为维度为128的嵌入向量（在原始的Llama 3 8B模型中，该维度为4096）。这些嵌入向量随后将被传递到下一个模块，即解码器模块。 

让我们开始构建输入块的第一个组件：分词器。

In [1]:
# Import necessary libraries
import torch
from torch import nn
from torch.nn import functional as F

import math
import numpy as np
import time
from dataclasses import dataclass
from typing import Optional, Tuple, List
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
print("PyTorch版本:", torch.__version__)
print("CUDA是否可用:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA版本:", torch.version.cuda)
    print("GPU设备:", torch.cuda.get_device_name(0))

PyTorch版本: 2.5.1+cu121
CUDA是否可用: True
CUDA版本: 12.1
GPU设备: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
import sys
import os


print("=== Python环境信息 ===")
print(f"Python可执行文件路径: {sys.executable}")
print(f"Python版本: {sys.version}")
print(f"当前工作目录: {os.getcwd()}")

print("\n=== PyTorch环境信息 ===")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"GPU设备: {torch.cuda.get_device_name(0)}")

=== Python环境信息 ===
Python可执行文件路径: d:\Hobby\Tools\anaconda3\envs\acp-gpu\python.exe
Python版本: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
当前工作目录: d:\Hobby\github\hobbytp.github.io\content\zh\base

=== PyTorch环境信息 ===
PyTorch版本: 2.5.1+cu121
CUDA是否可用: True
CUDA版本: 12.1
GPU设备: NVIDIA GeForce RTX 3050 Laptop GPU


In [18]:
def check_device():
    print("=== PyTorch 环境信息 ===")
    print(f"PyTorch 版本: {torch.__version__}")
    print(f"CUDA 是否可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA 版本: {torch.version.cuda}")
        print(f"当前 GPU: {torch.cuda.get_device_name(0)}")
    
    print("\n=== 设备分配测试 ===")
    # 测试方法1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"方法1 - 字符串方式:")
    print(f"设备: {device}")
    
    # 测试方法2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n方法2 - torch.device方式:")
    print(f"设备: {device}")
    
    # 创建测试张量
    print("\n=== 张量测试 ===")
    x = torch.randn(2, 3)
    print(f"初始张量设备: {x.device}")
    
    if torch.cuda.is_available():
        x = x.cuda()
        print(f"移动到CUDA后的设备: {x.device}")
        
        # 测试CUDA张量运算
        y = torch.randn(2, 3).cuda()
        z = x + y
        print(f"CUDA张量运算结果设备: {z.device}")


check_device()

=== PyTorch 环境信息 ===
PyTorch 版本: 2.6.0+cpu
CUDA 是否可用: False

=== 设备分配测试 ===
方法1 - 字符串方式:
设备: cpu

方法2 - torch.device方式:
设备: cpu

=== 张量测试 ===
初始张量设备: cpu


In [5]:
import sys
print(sys.executable)

d:\Hobby\Tools\anaconda3\envs\acp-gpu\python.exe


In [6]:
### Step 1: Input Block ###
# Using Tiny Shakespeare dataset for character-level tokenizer. Some part of the following character-level tokenizer is referenced from Andrej karpathy's GitHub (https://github.com/karpathy/nanoGPT/blob/master/data/shakespeare_char/prepare.py) which I found is explained very well.
# Load tiny_shakespeare data file (https://github.com/tamangmilan/llama3/blob/main/tiny_shakespeare.txt)

device: str = 'cuda' if torch.cuda.is_available() else 'cpu'   # Assign device to cuda or cpu based on availability

print(f"Using device: {device}")

# Load tiny_shakespeare data file.
with open('tiny_shakespeare.txt', 'r') as f:
  data = f.read()

# Prepare vocabulary by taking all the unique characters from the tiny_shakespeare data
vocab = sorted(list(set(data)))

# Training Llama 3 model requires addtional tokens such as <|begin_of_text|>, <|end_of_text|> and <|pad_id|>, we'll add them into vocabulary
vocab.extend(['<|begin_of_text|>','<|end_of_text|>','<|pad_id|>'])
vocab_size = len(vocab)

# Create a mapping between characters with corresponding integer indexes in vocabulary.
# This is important to build tokenizers encode and decode functions.
itos = {i:ch for i, ch in enumerate(vocab)}
stoi = {ch:i for i, ch in enumerate(vocab)}

# Tokenizers encode function: take a string, output a list of integers
def encode(s):
  return [stoi[ch] for ch in s]

# Tokenizers decode function: take a list of integers, output a string
def decode(l):
  return ''.join(itos[i] for i in l)

# Define tensor token variable to be used later during model training
token_bos = torch.tensor([stoi['<|begin_of_text|>']], dtype=torch.int, device=device)
token_eos = torch.tensor([stoi['<|end_of_text|>']], dtype=torch.int, device=device)
token_pad = torch.tensor([stoi['<|pad_id|>']], dtype=torch.int, device=device)

prompts = "Hello World"
encoded_tokens = encode(prompts)
decoded_text = decode(encoded_tokens)

### Test: Input Block Code ###
# You need take out the triple quotes below to perform testing

print(f"Lenth of shakespeare in character: {len(data)}")
"""
print(f"The vocabulary looks like this: {''.join(vocab)}\n")
print(f"Vocab size: {vocab_size}")
print(f"encoded_tokens: {encoded_tokens}")
print(f"decoded_text: {decoded_text}")
"""

Using device: cuda
Lenth of shakespeare in character: 1115394


'\nprint(f"The vocabulary looks like this: {\'\'.join(vocab)}\n")\nprint(f"Vocab size: {vocab_size}")\nprint(f"encoded_tokens: {encoded_tokens}")\nprint(f"decoded_text: {decoded_text}")\n'

# 步骤2：解码器块 

如果你查看上面的架构图，解码器块包含以下子组件。
- 均方根归一化（RMS Norm）
- 旋转位置编码（Rotary Positional Encoding）
- KV 缓存（KV Cache）
- 分组查询注意力（Group Query Attention）
- 前馈网络（FeedForward Network）
- 解码器块 
让我们逐一深入了解这些子组件。

## 2a. RMSNorm（均方根归一化）

为何需要RMSNorm？ 在上文架构图中，您可能已注意到输入块（即嵌入向量）的输出会经过RMSNorm模块。这是因为嵌入向量通常具有高维度（如Llama3-8B模型的4096维），且各维度数值范围可能存在显著差异。这种差异会导致模型梯度出现爆炸或消失问题，进而引发收敛速度缓慢甚至训练发散的风险。
RMSNorm通过将数值约束到特定范围内，有效稳定并加速训练过程。其核心作用是使梯度幅值保持相对一致，从而显著提升模型的收敛效
率。


RMSNorm如何工作？ 让我们先看下图解说明。

![](images/rms_norm.webp)
[作者图片]：形状为 [3,3] 的输入嵌入上的 RMSNorm 实现

与层归一化（Layer Normalization）类似，RMSNorm同样沿嵌入特征或维度进行归一化处理。 上图中嵌入向量的形状为 3,3，表示每个词元（token）具有3个维度。


**示例：** 让我们对第一个词元X1的嵌入向量应用RMSNorm：  

词元X1在每个维度的值（即x₁₁、x₁₂和x₁₃）将分别除以这些值的**均方根（Root Mean Square）**。计算公式如上图所示。  

- **数值稳定性处理**：在均方根中添加极小常数ε（Epsilon），避免除以零的情况。  
- **可学习参数Gamma（γ）**：最终乘以一个缩放参数γ。每个特征维度拥有独立的γ参数（如上图中维度d₁对应γ₁，d₂对应γ₂，d₃对应γ₃）。γ作为可训练参数，通过动态调整进一步增强归一化的稳定性，其初始值通常设为1（如示例计算所示）。  

**效果对比**：  
从示例可见，原始嵌入值范围较大且分布分散，而经RMSNorm处理后，数值范围显著缩小且分布集中。上述计算严格遵循RMSNorm的数学定义实现。  

**为何选择RMSNorm而非层归一化（Layer Normalization）？**

通过前文示例可见，RMSNorm无需计算均值（mean）和方差（variance）——这两项是层归一化的核心计算步骤。因此，RMSNorm通过省略均值与方差的计算，显著降低了运算开销。

此外，根据原论文作者的实验验证，RMSNorm在保持模型精度的前提下，还能提供额外的性能优势（如训练速度提升或资源消耗减少）。

让我们来编写RMSNorm代码：

In [7]:
# Step2: The Decoder Block
# 注意：由于Llama 3模型由Meta开发，为保持与其代码库同步并确保未来兼容性，
# 我们将主要基于Meta GitHub的代码进行必要修改以实现目标。

# 定义参数数据类：这些参数将用于模型构建、训练和推理阶段。
# 注意：由于当前目标侧重于快速验证训练/推理结果而非追求高精度，
# 我们为多数参数设置了低于Llama 3原模型的默认值（原模型参数值更大）。

@dataclass
class ModelArgs:
    dim: int = 512              # embedding dimension
    n_layers: int = 8           # number of model decoder blocks
    n_heads: int = 8            # number of heads for queries embedding
    n_kv_heads: int = 4         # number of heads for keys and values embedding
    vocab_size: int = len(vocab) # Length of vocabulary
    multiple_of: int = 256        # Require to calculate dim of feedfoward network
    ffn_dim_multiplier: Optional[float] = None  # Require to calculate dim of feedfoward network
    norm_eps: float = 1e-5                       # Default Epsilon value set for the RMSNorm calculation
    rope_theta: float = 10000.0   # Default theta value for the RePE calculation

    max_batch_size: int = 10     # Max batch size
    max_seq_len: int = 256         # Max sequence length

    epochs: int = 2500             # Total number of training iteration
    log_interval: int = 10        # Number of interval to print the logs and loss values   
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'   # Assign device to cuda or cpu based on availability 

In [8]:
## Step2a: The RMSNorm

class RMSNorm(nn.Module):
  def __init__(self, dim: int, eps: float = 1e-6):
    super().__init__()
    device = ModelArgs.device
    self.eps = eps
    # Scaling parameter gamma, initialized with one and the no of parameters is equal to the size of dim
    #定义一个可学习的参数，初始化为全 1 的张量，形状为 (dim,)，并将其放到指定设备上。该参数用于对归一化后的结果进行缩放。
    self.weight = nn.Parameter(torch.ones(dim).to(device))

  def _norm(self, x):
    # _norm 是一个私有方法，用于执行 RMS 归一化操作。与 LayerNorm 的区别：RMSNorm 省略了均值中心化（只缩放不平移），计算更高效。
    # x.pow(2)：对输入张量 x 的每个元素进行平方运算。
    # mean(dim=-1, keepdim=True)：沿着最后一个维度计算平方后元素的均值，keepdim=True 保证计算后维度不变。
    # torch.rsqrt：计算均值的倒数平方根（即 1 / sqrt(mean + eps)），等价于标准差归一化的变体。这里计算均值加上 self.eps 后的结果的倒数平方根。
    # 最后将输入张量 x 乘以这个倒数平方根，得到归一化后的张量，并将其放到指定设备上，确保与其他张量在同一设备上运算。
    return x * torch.rsqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps).to(device)

  def forward(self, x):
    """
    
    """
    #Shape: x[bs,seq,dim]
    #说明输入张量x的预期形状：[batch_size, sequence_length, feature_dimension]
    # 输入张量 x 的形状为 [batch_size, sequence_length, feature_dimension]，表示一个批次的输入数据，其中 batch_size 是样本数量，sequence_length 是序列长度，feature_dimension 是特征维度。
    # x.float(): 将输入转换为浮点型（确保兼容性）
    # self._norm(): 调用内部归一化层
    # .type_as(x): 将结果转换回输入x的原始数据类型
    # 输出形状保持不变 [bs, seq, dim]
    output = self._norm(x.float()).type_as(x)

    #Shape: x[bs,seq,dim] -> x_norm[bs,seq,dim]
    # 将归一化后的输出与可学习的权重self.weight相乘（逐元素乘法）
    # self.weight是可训练参数，形状应为[dim]，PyTorch会自动广播到输入形状
    return output * self.weight

### Test: RMSNorm Code ###
# You need take out the triple quotes below to perform testing
"""
x = torch.randn((ModelArgs.max_batch_size, ModelArgs.max_seq_len, ModelArgs.dim), device=device)
rms_norm = RMSNorm(dim=ModelArgs.dim)
x_norm = rms_norm(x)

print(f"Shape of x: {x.shape}")
print(f"Shape of x_norm: {x_norm.shape}")
"""
### Test Results: ###
"""
Shape of x: torch.Size([10, 256, 512])
Shape of x_norm: torch.Size([10, 256, 512])
"""

'\nShape of x: torch.Size([10, 256, 512])\nShape of x_norm: torch.Size([10, 256, 512])\n'

In [10]:
# 上面RMSNorm的测试代码
x = torch.randn((ModelArgs.max_batch_size, ModelArgs.max_seq_len, ModelArgs.dim), device=device)
rms_norm = RMSNorm(dim=ModelArgs.dim)
x_norm = rms_norm(x)

print(f"Shape of x: {x.shape}")
print(f"Shape of x_norm: {x_norm.shape}")

Shape of x: torch.Size([10, 256, 512])
Shape of x_norm: torch.Size([10, 256, 512])



## 2b. 旋转位置编码（RoPE）

为什么我们需要旋转位置编码（RoPE）？在解释原因之前，先回顾一下我们目前的流程：首先，我们已经将输入文本转换成了嵌入向量；接着，对嵌入向量应用了RMSNorm。此时你可能已经注意到一个问题：假如输入文本是“I love apple”或“apple love I”，模型依然会将这两句话当作相同的内容来学习，因为在嵌入向量中并没有定义顺序信息。**因此，顺序对于任何语言模型来说都至关重要**。在Llama 3的模型架构中，RoPE被用来为每个词元定义在句子中的位置，不仅保证了顺序，还能维护句子中词元之间的相对位置关系。

那么，什么是旋转位置编码（RoPE），它是如何工作的？  
如上文所述，RoPE是一种位置编码方法，它通过为嵌入向量添加绝对位置信息，以及融合词元之间的相对位置信息，从而保持句子中词元的顺序。其核心机制是：**通过一个被称为“旋转矩阵”的特殊矩阵，对给定的嵌入向量进行旋转变换**。这种基于旋转矩阵的简洁而强大的数学推导，就是RoPE的本质。

![RoPE](images/rope.webp)
[作者图片]：旋转位置编码（RoPE）的数学推导

这张图非常直观地展示了**旋转矩阵（Rotation Matrix）**在二维向量空间中的作用，是理解RoPE（旋转位置编码）数学本质的关键。

**图片内容详解：**
1. 向量定义
- **Q1(x1, y1)：**
  - 一个二维查询向量（Query Vector），模长为d，与x轴夹角为θ1。
- **Q2(x2, y2)：**
  - 由Q1顺时针旋转θ2角度得到的新向量，模长仍为d。

2. 旋转过程
- **Q1 → Q2的变换：**
  - Q2是通过对Q1应用旋转操作得到的，旋转角度为θ2。
  - 旋转后，向量的长度不变，只改变了方向。

3. 数学表达
- **极坐标表示：**
  - Q1和Q2都可以用极坐标（模长d和角度）表示。
- **旋转矩阵公式：**  
  - 旋转后的新坐标（Q2）可以通过如下矩阵乘法得到：  
$$
\begin{bmatrix}
x_2 \\
y_2
\end{bmatrix}
=
\begin{bmatrix}
\cos\theta_2 & -\sin\theta_2 \\
\sin\theta_2 & \cos\theta_2
\end{bmatrix}
\begin{bmatrix}
x_1 \\
y_1
\end{bmatrix}
$$
  - 其中，旋转矩阵的每个元素都与旋转角度θ2相关。

4. 物理意义
- **旋转矩阵的作用：**
  - 保持向量模长不变，仅改变方向。
  - 本质上是将原始向量在二维平面上“旋转”一个角度。

**与RoPE的关系**

- **RoPE的核心思想**就是利用这种旋转矩阵，把每一个token的嵌入向量在高维空间中按照其序列位置做“旋转”。
- 这样做能让模型既感知到绝对位置信息，也能自然编码相对位置信息（即token之间的距离/顺序）。


所以，上图中的旋转矩阵用于对二维向量进行旋转。然而，Llama 3模型中的向量维度高达4096，远远超过二维空间。那么，我们该如何将旋转操作应用到高维嵌入向量上呢？接下来让我们详细探讨高维嵌入空间中的旋转方法。

![RoPE embedding](images/rope_embedding.webp)

我们现在已经知道，嵌入向量的旋转涉及到对每个嵌入位置 \(m\) 的数值与每对嵌入维度对应的角度 \(\theta\) 相乘。这正是RoPE（旋转位置编码）通过旋转矩阵实现对绝对位置信息和相对位置信息共同建模的原理。

注意事项：
- 在进行旋转之前，需要将旋转矩阵转换为极坐标形式，并将嵌入向量转换为复数表示。
- 旋转操作完成后，需要将旋转后的嵌入向量重新转换回实数形式，以便用于注意力机制的后续计算。
- RoPE仅应用于Query和Key的嵌入向量，不作用于Value嵌入。

让我们深入探讨RoPE编码：

### GPT4.1 解释下面代码：  
这段代码是**RoPE（Rotary Position Embedding）旋转位置编码**的核心实现。它在Transformer模型中非常流行，特别是在Llama、ChatGLM等大模型中。下面我用通俗易懂的语言，逐步为您拆解每个函数的作用和背后的原理。

#### 1. `precompute_freqs_cis`

**作用**：  
为每对维度（比如一个token的128维embedding，有64对维度）生成一组“旋转频率”，并把它们变成可以直接用于复数旋转的形式（极坐标复数），以便后续直接对embedding做旋转。每对维度对应一个旋转频率，频率越高，旋转越快。

**通俗解释**：

- **每对维度一个频率**：比如128维的embedding，会有64个不同频率，每对（第0、1维，第2、3维...）用一个频率。
- **freqs**：生成一个形如[1, θ^(-2/dim), θ^(-4/dim), ...]的频率列表（其实是倒数）。
- **t**：表示token在序列中的位置（0, 1, 2...）。
- **torch.outer(t, freqs)**：把所有位置和所有频率组合起来，得到一个二维矩阵，每一行对应一个token位置，每一列对应一个频率。
- **torch.polar**：把每个位置-频率对变成单位复数（极坐标形式），这样可以表示二维平面的旋转。

**一句话总结**：  
给每个token的每对维度，预生成一个“旋转角度”，用复数来表示。

#### 2. `reshape_for_broadcast`

**作用**：  
把`freqs_cis`的形状变成可以和`xq`、`xk`做逐元素乘法（广播机制）。

**通俗解释**：

- 由于后面要用`freqs_cis`和query/key做乘法，形状要对齐。这个函数就是把`freqs_cis`变成和输入tensor兼容的形状。
- 比如：原本`freqs_cis`形状是[seq_len, head_dim]，要变成[1, seq_len, 1, head_dim]，这样可以和[batch, seq_len, n_heads, head_dim]的embedding对齐。

#### 3. `apply_rotary_emb`

**作用**：  
对query和key的embedding做“旋转位置编码”，让模型感知位置信息。

**通俗解释**：

1. **重组维度**：  
   - 把最后一维（head_dim）两两分组，变成复数（实部+虚部）。这样每对维度就能看作二维平面上的一个点。
2. **变成复数**：  
   - 用`torch.view_as_complex`，把每对维度变成一个复数，方便后续做旋转。
3. **准备旋转因子**：  
   - 调用`reshape_for_broadcast`，让旋转因子`freqs_cis`和embedding形状对齐。
4. **旋转**：  
   - 用复数乘法，实现二维平面的旋转（这就是RoPE的核心！），本质上就是给每对维度加上一个“角度”。
5. **还原回实数**：  
   - 把旋转后的复数结果拆回实部和虚部，展平成原来的维度。

**一句话总结**：  
把embedding的每对维度当作平面上的点，给它们加上一个“旋转角度”，这样模型就能区分不同token的位置。

#### 4. **整体流程可视化**

1. 对每对维度，预先计算好每个位置的旋转角度（用复数表示）。
2. 把embedding的每对维度变成复数（二维点）。
3. 用复数乘法，对每个token的每对维度做旋转。
4. 旋转后的结果再拆成实部和虚部，拼回原始embedding形状。



下面用表格方式解释函数 precompute_freqs_cis的步骤：

| 步骤         | 作用                           | 例子/解释                          |
|--------------|------------------------------|------------------------------------|
| 频率生成     | 每对维度一个频率               | 频率慢→慢转，频率快→快转           |
| 位置索引     | 每个token一个位置编号           | 0, 1, 2, ..., seq_len-1            |
| 角度矩阵     | 每个位置×每个频率得一个角度     | 角度 = 位置 × 频率                 |
| 极坐标转复数 | 角度转成可用的旋转因子          | cosθ + i·sinθ                      |


In [ ]:
## Step2b: The RoPE
def precompute_freqs_cis(dim:int, seq_len: int, theta: float=10000.0):
  """
  作用是预先计算旋转位置编码（RoPE）所需的复指数（cis）频率。
  这个函数是用于实现 RoPE（旋转位置编码，Rotary Positional Embedding） 的关键部分。
  这个函数的作用是预计算一组复数旋转因子，这些因子会被用来给神经网络的嵌入（embedding）加上位置信息，帮助模型理解不同token在序列中的顺序。
  
  RoPE 是一种将位置信息融入注意力机制的方法，通过复数旋转来实现。
  - dim：输入特征的维度，通常是注意力头的维度。
  - seq_len：序列的最大长度。
  - theta：频率缩放因子，默认值为 10000.0。
  
  该函数通过计算频率值、序列位置，然后求外积并转换为极坐标形式，得到用于旋转位置编码的复指数频率张量。
  这个张量会在后续的 apply_rotary_emb 函数中与查询和键向量相乘，将位置信息融入到注意力机制中。
  """
  # Computing Theta value for each dim pair which is dim/2
  device = ModelArgs.device
  # torch.arange(0, dim, 2, device=device)[:(dim//2)].float()：生成一个从 0 开始，步长为 2，长度为 dim//2 的张量（因为RoPE是两两一组处理的），然后转换为浮点数类型。
  # theta ** (...)：将 theta 进行幂运算。
  # 1.0 / (...)：计算幂运算结果的倒数，得到频率值 freqs。
  # freqs = 1.0 / (theta ** (序列/总维度))，这样每对维度都有一个不同的“频率”，这个频率决定了旋转的快慢。
  freqs = 1.0 / (theta ** (torch.arange(0, dim, 2,device=device)[:(dim//2)].float()/dim))

  # Computing range of positions(m) in the sequence
  # 生成一个从 0 到 seq_len - 1 的浮点数张量 t，表示序列中每个位置的索引，代表每个token在句子里的位置。
  t = torch.arange(seq_len, dtype=torch.float32, device=device)

  # freqs gives all the Theta value range for all the position of tokens in the sequence
  # 计算 t 和 freqs 的外积，得到一个形状为 (seq_len, dim//2) 的张量。这个张量包含了序列中每个位置对应每个频率的值。
  # torch.outer(t, freqs) 计算每个token位置和每个维度频率的乘积，得到一个二维矩阵，每个元素代表“第t个token在第d对维度上的旋转角度”。
  freqs = torch.outer(t, freqs).to(device)

  # This is the rotation matrix which needs to be converted to Polar form in order to perform rotation to the embedding
  # torch.ones_like(freqs).to(device)：生成一个与 freqs 形状相同且元素全为 1 的张量。
  # torch.polar(...)：将幅度（全 1 张量）和相位（freqs）转换为复数的极坐标形式，得到复指数频率 freqs_cis。
  # torch.polar(torch.ones_like(freqs), freqs) 这一步把每个角度变成单位模长的复数（即cos(θ) + i*sin(θ)），这样后面可以直接用复数乘法来实现旋转。
  freqs_cis = torch.polar(torch.ones_like(freqs).to(device), freqs).to(device)
  # 返回预先计算好的复指数频率 freqs_cis，用于后续的旋转位置编码操作。
  # 返回的 freqs_cis 就是所有位置、所有维度对的旋转因子。
  return freqs_cis

def reshape_for_broadcast(freqs_cis, x):
  """
    该函数其核心目的是对 freqs_cis 张量进行形状重塑，使其能够与输入张量 x 进行广播操作。
    具体来说，它将 freqs_cis 张量的形状调整为除第 1 维和最后一维外，其余维度大小均为 1 的形状。
    广播操作允许不同形状的张量在进行逐元素运算时，自动扩展维度以匹配形状。

    Args:
        freqs_cis：需要进行形状重塑的张量，代表旋转位置编码所需的旋转矩阵。
        x：输入张量，例如查询（query）和键（key）向量，后续会与 freqs_cis 进行逐元素乘法运算。
    Returns:
        _type_: _description_
  """
  # 用于获取输入张量 x 的维度数，
  ndim = x.ndim
  # assert 0<=1<ndim
  assert ndim >= 2, "Input tensor must have at least 2 dimensions"
  
  # 频率张量freqs_cis的形状必须与输入张量x的序列长度和特征维度完全匹配
  assert freqs_cis.shape == (x.shape[1],x.shape[-1]), "the last two dimension of freqs_cis, x must match"
  
  # 这里使用列表推导式生成一个新的形状列表 shape。
  # 对于 x 的每个维度，若维度索引 i 为 1 或者是最后一维（i == ndim - 1），则取 x 对应维度的大小 d；
  # 否则，该维度大小设为 1。这样做的目的是让 freqs_cis 除了第 1 维和最后一维外，其他维度都扩展为 1，以便与 x 进行广播操作。
  shape = [d if i==1 or i==ndim-1 else 1 for i,d in enumerate(x.shape)]
  
  # view 是 PyTorch 张量的方法，用于对 freqs_cis 张量进行形状重塑，使其形状变为 shape 列表所指定的形状。
  # *shape 是解包操作，将 shape 列表中的元素依次传递给 view 方法。
  return freqs_cis.view(*shape)

def apply_rotary_emb(xq: torch.Tensor, xk: torch.Tensor, freqs_cis: torch.Tensor)->Tuple[torch.Tensor, torch.Tensor]:
  device = ModelArgs.device
  # Applying rotary positional encoding to both query and key embedding together
  # First: The last dimension of xq and xk embedding needs to be reshaped to make it a pair. As rotation matrix is applied to each pair of dim.
  # Next: convert both xq and xk to complex number as the rotation matrix is only applicable to complex number
  xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2)).to(device)    #xq_:[bsz, seq_len, n_heads, head_dim/2]
  xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2)).to(device)    #xk_:[bsz, seq_len, n_heads, head_dim/2]

  # The rotation matrix(freqs_cis) dimensions across seq_len(dim=1) and head_dim(dim=3) should match with the embedding
  # Also, the shape freqs_cis should be the same with xq and xk, hence change the shape of freqs_cis:[seq_len,head_dim] -> freqs_cis:[1,seq_len,1,head_dim]
  freqs_cis = reshape_for_broadcast(freqs_cis, xq_)

  #Finally, perform rotation operation by multiplying with freqs_cis.
  #After the rotation is completed, convert both xq_out and xk_out back to real number and return
  xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3).to(device) #xq_out:[bsz, seq_len, n_heads, head_dim]
  xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3).to(device) #xk_out:[bsz, seq_len, n_heads, head_dim]
  return xq_out.type_as(xq), xk_out.type_as(xk)

### Test: RoPE Code ###
# Note: x_norm is calculated during RMSNorm and is being used for testing here.
# You need take out the triple quotes below to perform testing
"""
head_dim = ModelArgs.dim//ModelArgs.n_heads
wq = nn.Linear(ModelArgs.dim, ModelArgs.n_heads * head_dim, bias=False, device=device)
wk = nn.Linear(ModelArgs.dim, ModelArgs.n_kv_heads * head_dim, bias=False, device=device)
xq = wq(x_norm)
xk = wk(x_norm)
print(f"xq.shape: {xq.shape}")
print(f"xk.shape: {xk.shape}")

xq = xq.view(xq.shape[0],xq.shape[1],ModelArgs.n_heads, head_dim)
xk = xk.view(xk.shape[0],xk.shape[1],ModelArgs.n_kv_heads, head_dim)
print(f"xq.re-shape: {xq.shape}")
print(f"xk.re-shape: {xk.shape}")

freqs_cis = precompute_freqs_cis(dim=head_dim, seq_len=ModelArgs.max_seq_len)
print(f"freqs_cis.shape: {freqs_cis.shape}")

xq_rotate, xk_rotate = apply_rotary_emb(xq, xk, freqs_cis)
print(f"xq_rotate.shape: {xq_rotate.shape}")
print(f"xk_rotate.shape: {xk_rotate.shape}")
"""
### Test Results: ###
"""
xq.shape: torch.Size([10, 256, 512])
xk.shape: torch.Size([10, 256, 256])
xq.re-shape: torch.Size([10, 256, 8, 64])
xk.re-shape: torch.Size([10, 256, 4, 64])
freqs_cis.shape: torch.Size([256, 32])
xq_rotate.shape: torch.Size([10, 256, 8, 64])
xk_rotate.shape: torch.Size([10, 256, 4, 64])
"""

## 2c. KV Cache（仅在推理时需要）：
什么是 KV-Cache？在 Llama 3 架构中，推理阶段引入了 KV-Cache（键值缓存）的概念，用于以 Key 和 Value 的形式存储先前生成的 token。这些缓存会被用于后续的自注意力计算，从而生成下一个 token。这里只缓存了 key 和 value，而没有缓存 query，因此称之为 KV Cache。

\begin{align*}
Q &= XW^Q \\
K &= XW^K \\
V &= XW^V \\
\end{align*}

如果是多头注意力（Multi-Head Attention），会有多个 Q/K/V 权重矩阵，分别生成多个头，然后拼接或投影回原空间。每个 head 都有自己的WQ, WK, WV.

为什么需要 KV Cache？
- 推理时的缓存：在生成文本的过程中，模型每次只需要生成下一个token。每一步都要计算自注意力（Self-Attention），而自注意力需要访问所有历史token的Key和Value。
- 缓存机制：为了避免重复计算历史token的Key和Value，模型会在每一步把已经生成的Key和Value缓存下来（即KV-Cache）。
- 高效推理：下一个token推理时，只需为新token计算Query，并与之前所有缓存好的Key和Value进行注意力计算，而不用重新回头计算所有历史token的K和V。
- 为什么只缓存K和V？
  - Query是针对当前生成token实时计算的，每步都不同且只需当前步用一次；而Key和Value是历史token的“静态”表示，可以复用，所以只缓存K和V，这就是“KV-Cache”名称的由来。
  - Key和Value通过线性变换从输入向量获得，参与自注意力的加权计算。
  - KV-Cache极大加速了长序列推理，是大模型高效生成的关键技术之一。


让我们通过下面的图示进一步澄清这个问题。
![](images/kv_cache.webp)

在图示的A模块中，当生成output3 token时，之前的输出token（output1、output2）依然会被重新计算，这实际上是完全没有必要的。这导致在注意力计算过程中出现了额外的矩阵乘法，从而大幅增加了计算资源的消耗。

而在图示的B模块中，输出token会替换输入token作为Query embedding，KV Cache则负责存储之前生成的token。在计算注意力得分时，我们只需要用当前的一个Query token，再结合Key和Value缓存中的历史token即可。这样，矩阵乘法的规模从A模块的3x3降到了B模块的1x3，计算量减少了近66%。在实际应用中，随着序列长度和batch size的增大，这种优化能显著降低算力消耗。最终，每次推理时只会生成一个最新的输出token，这正是引入KV-Cache的核心原因。


## 2d. 分组查询注意力（Group Query Attention）：
分组查询注意力（Group Query Attention，简称GQA）与之前模型（如Llama 1）中使用的多头注意力（Multi-Head Attention）基本相同，唯一的区别在于对查询（Query）和键/值（Key/Value）分别使用了不同的头。通常，分配给查询的头数是键和值头数的n倍。我们可以结合下方的图示进一步加深理解。
![](images/gqa.webp)

在给定的图示中，多头注意力（Multi-Head Attention）为所有的查询（Query）、键（Key）和值（Value）分配了相同数量的头，即 n_heads = 8。

而在分组查询注意力（Group Query Attention）模块中，查询有8个头（n_heads），而键和值只有4个头（n_kv_heads），也就是查询头数的二分之一。

既然Multi-Head Attention已经很优秀了，为什么还需要Group Query Attention？要回答这个问题，我们需要回到KV Cache。KV Cache极大地减少了计算资源消耗，但随着缓存的token越来越多，内存资源的占用也会显著增加。这对于模型性能和成本来说都不是好事。因此，引入了Group Query Attention。通过减少K和V的头数，降低了需要存储的参数数量，从而减少了内存消耗。多项测试结果表明，这种做法下模型的准确率基本保持不变。

让我们在代码中实现这个。

In [ ]:
## The Attention Block [Step2c: The KV Cache; Step2d: Group Query Attention]
## As mentioned before, the naming convention follows original the meta's LLama3 GitHub

class Attention(nn.Module):
  def __init__(self, args: ModelArgs):
    super().__init__()
    self.args = args
    # Embedding dimension
    self.dim = args.dim
    # Number of heads assigned to Query
    self.n_heads = args.n_heads
    # Number of heads assigned to Key and values. If "None", the number will be same as Query.
    self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
    # Dimension of each head relative to model dimension
    self.head_dim = args.dim // args.n_heads
    # Number of repetition in order to make time Key, Value heads to match Query heads number
    self.n_rep = args.n_heads // args.n_kv_heads

    # Weight initialize for Keys, Querys, Values and Oupt. Notice that the out_feature value of weight for q and kv are based on it's heads
    self.wq = nn.Linear(self.dim, self.n_heads * self.head_dim, bias=False, device=device)
    self.wk = nn.Linear(self.dim, self.n_kv_heads * self.head_dim, bias=False, device=device)
    self.wv = nn.Linear(self.dim, self.n_kv_heads * self.head_dim, bias=False, device=device)
    self.wo = nn.Linear(self.n_heads * self.head_dim, self.dim, bias=False, device=device)

    # Initialize caches to store Key, Values at start. (KV Cache Implementation)
    self.cache_k = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim), device=args.device)
    self.cache_v = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim), device=args.device)

  def forward(self, x: torch.Tensor, start_pos, inference):
    # Shape of the input embedding: [bsz,seq_len,dim]
    bsz, seq_len, _ = x.shape
    # Mask will be used during 'Training' and is not required for 'inference' due to the use of KV cache.
    mask = None

    xq = self.wq(x)  #x[bsz,seq_len,dim]*wq[dim,n_heads * head_dim] -> q[bsz,seq_len,n_heads * head_dim]
    xk = self.wk(x)  #x[bsz,seq_len,dim]*wq[dim,n_kv_heads * head_dim] -> k[bsz,seq_len,n_kv_heads * head_dim]
    xv = self.wv(x)  #x[bsz,seq_len,dim]*wq[dim,n_kv_heads * head_dim] -> v[bsz,seq_len,n_kv_heads * head_dim]

    # Reshaping Querys, Keys and Values by their number of heads. (Group Query Attention Implementation)
    xq = xq.view(bsz, seq_len, self.n_heads, self.head_dim)      #xq[bsz,seq_len,n_heads, head_dim]
    xk = xk.view(bsz, seq_len, self.n_kv_heads, self.head_dim)   #xk[bsz,seq_len,n_kv_heads, head_dim]
    xv = xv.view(bsz, seq_len, self.n_kv_heads, self.head_dim)   #xv[bsz,seq_len,n_kv_heads, head_dim]

    # Model - Inference Mode: kv-cache is enabled at inference mode only.
    if inference:
      # Compute rotation matrix for each position in the sequence
      freqs_cis = precompute_freqs_cis(dim=self.head_dim, seq_len=self.args.max_seq_len * 2)
      # During inferencing, we should only take the rotation matrix range from the current position of the tokens.
      freqs_cis = freqs_cis[start_pos : start_pos + seq_len]
      # Apply RoPE to Queries and Keys embeddings
      xq, xk = apply_rotary_emb(xq, xk, freqs_cis)

      self.cache_k = self.cache_k.to(xq)
      self.cache_v = self.cache_v.to(xq)
      # Store Keys and Values token embedding into their respective cache [KV Cache Implementation]
      self.cache_k[:bsz, start_pos:start_pos + seq_len] = xk
      self.cache_v[:bsz, start_pos:start_pos + seq_len] = xv

      # Assign all the previous tokens embeddings upto current tokens position to Keys and Values variable for Attention Calculation
      keys = self.cache_k[:bsz, :start_pos + seq_len]
      values = self.cache_v[:bsz, :start_pos + seq_len]

      # At this point, they Keys and Values shape aren't same with Queries Embedding which has to be in order to computer attention score
      # Use repeat_kv function to make Keys,Values shape same as queries shape
      keys = repeat_kv(keys, self.n_rep)      #keys[bsz,seq_len,n_heads,head_dim]
      values = repeat_kv(values, self.n_rep)  #values[bsz,seq_len,n_heads,head_dim]

    # Mode - Training mode: KV-Cache not implemented
    else:
      # Compute rotation matrix and apply RoPE to queries and keys for for training.
      freqs_cis = precompute_freqs_cis(dim=self.head_dim, seq_len=self.args.max_seq_len)

      #xq[bsz,seq_len,n_heads, head_dim], xk[bsz,seq_len,n_heads, head_dim]
      xq, xk = apply_rotary_emb(xq, xk, freqs_cis)

      # Use repeat_kv function to make Keys,Values shape same as the queries shape
      #keys[bsz,seq_len,n_heads,head_dim], #values[bsz,seq_len,n_heads,head_dim]
      keys = repeat_kv(xk, self.n_rep)
      values = repeat_kv(xv, self.n_rep)

      # For training mode, we'll compute mask and apply to the attention score later
      mask = torch.full((seq_len, seq_len),float("-inf"),device=self.args.device)
      mask = torch.triu(mask, diagonal=1).to(self.args.device)

    # To compute attention, we'll need to perform a transpose operation to reshape all queries, keys and values bring heads at dim 1 and seq at dim 2
    xq = xq.transpose(1,2)                  #xq[bsz,n_heads,seq_len,head_dim]
    keys = keys.transpose(1,2)              #keys[bsz,n_heads,seq_len,head_dim]
    values = values.transpose(1,2)          #values[bsz,n_heads,seq_len,head_dim]

    # Computing attention score
    scores = torch.matmul(xq, keys.transpose(2,3)).to(self.args.device)/math.sqrt(self.head_dim)
    if mask is not None:
      scores = scores + mask

    # Apply softmax to the attention score
    scores = F.softmax(scores.float(), dim=-1).type_as(xq)
    # Matrix multiplication of attention score with the values
    output = torch.matmul(scores, values).to(self.args.device)

    # We get the contextual embedding for each head
    # All heads need to be reshaped back and combined to give a single single contextual attention output
    # Shape change: output[bsz,n_heads,seq_len,head_dim] -> output[bsz,seq_len, n_heads,head_dim] -> output[bsz,seq_len, n_heads * head_dim]
    output = output.transpose(1,2).contiguous().view(bsz, seq_len, -1)

    # shape: output [bsz,seq_len,dim]
    return self.wo(output)

# If the number of keys/values heads is less than query heads, this function expands the key/values embeddings with the required number of repetition
def repeat_kv(x:torch.Tensor, n_rep: int)-> torch.Tensor:
  bsz, seq_len, n_kv_heads, head_dim = x.shape
  if n_rep == 1:
    return x
  return (
      x[:,:,:,None,:]
      .expand(bsz,seq_len,n_kv_heads,n_rep, head_dim)
      .reshape(bsz,seq_len,n_kv_heads * n_rep, head_dim)
  )


### Test: Repeat_kv function ###
# note: xk, x_norm is already calculated during RoPE, RMSNorm testing and is being used for testing here.
# You need take out the triple quotes below to perform testing
"""
n_rep = ModelArgs.n_heads // ModelArgs.n_kv_heads
keys = repeat_kv(xk, n_rep)
print(f"xk.shape: {xk.shape}")
print(f"keys.shape: {keys.shape}")

## Test: Attention function
# You need take out the triple quotes below to perform testing

attention = Attention(ModelArgs)
x_out = attention(x_norm,start_pos=0, inference=False)
print(f"x_out.shape: {x_out.shape}")
"""
### Test Results: ###
"""
xk.shape: torch.Size([10, 256, 4, 64])
keys.shape: torch.Size([10, 256, 8, 64])
x_out.shape: torch.Size([10, 256, 512])
"""

## 2e. 前馈神经网络（SwiGLU激活函数）：
前馈神经网络在解码器模块中起什么作用？

如上方架构图所示，注意力机制的输出首先经过RMSNorm归一化，然后输入到前馈神经网络（FeedForward Network）。在前馈网络内部，注意力输出的嵌入（embedding）会在隐藏层中被扩展到更高的维度，从而学习token更复杂的特征。

为什么要用SwiGLU而不是ReLU？

我们可以结合下方的图示来寻找答案。

![swiglu](images/swiglu.webp)

如上图所示，SwiGLU激活函数在正轴上的表现几乎与ReLU相同。但在负轴上，SwiGLU会输出一些负值，而不是像ReLU那样直接输出0。这种特性有助于模型在负区间学习到更细微的特征，而不是一刀切地归零。总体而言，据作者所述，SwiGLU在实际表现上优于ReLU，因此被选用。

接下来，让我们深入了解前馈网络（FeedForward）的代码实现：




In [ ]:
## Step2e: The Feedfoward Network (SwiGLU activation)
class FeedForward(nn.Module):
  def __init__(self, dim:int, hidden_dim:int, multiple_of:int, ffn_dim_multiplier: Optional[float]):
    super().__init__()
    # Models embedding dimension
    self.dim = dim

    # We must use the hidden dimensions calculation shared by Meta which is the ideal one for this model
    # Hidden dimension are calculated such that it is a multiple of 256.
    hidden_dim = int(2 * hidden_dim/3)
    if ffn_dim_multiplier is not None:
      hidden_dim = int(ffn_dim_multiplier * hidden_dim)
    hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

    # define hiddne layers weights
    self.w1 = nn.Linear(self.dim, hidden_dim, bias=False, device=device)
    self.w2 = nn.Linear(hidden_dim, self.dim, bias=False, device=device)
    self.w3 = nn.Linear(self.dim, hidden_dim, bias=False, device=device)

  def forward(self, x):
    # Shape: [bsz,seq_len,dim]
    return self.w2(F.silu(self.w1(x)) * self.w3(x))



### Test: FeedForward module ###
# note: x_out is already computed at Attention testing and is being used for testing here.
# You need take out the triple quotes below to perform testing
"""
feed_forward = FeedForward(ModelArgs.dim, 4 * ModelArgs.dim, ModelArgs.multiple_of, ModelArgs.ffn_dim_multiplier)
x_out = rms_norm(x_out)
x_out = feed_forward(x_out)
print(f"feed forward output: x_out.shape: {x_out.shape}")
"""

### Test Results: ###
"""
feed forward output: x_out.shape: torch.Size([10, 256, 512])
"""

## 2f. 解码器模块（Decoder Block）：

如上方架构图（最开始的那张图）所示，解码器模块由多个子组件组成，这些组件我们在前面的2a到2f小节已经学习并实现过。下面是解码器模块内部逐步执行的操作流程：

1. **输入嵌入**（embedding）首先被送入Attention-RMSNorm模块，然后进一步输入到Group Query Attention模块。
2. **同样的输入嵌入**会与注意力输出（attention output）进行相加。
3. **相加后的注意力输出**再被送入FeedForward-RMSNorm模块，随后进入前馈神经网络（FeedForward network）模块。
4. **前馈网络的输出**会再次与注意力输出相加。
5. **最终的结果**被称为解码器输出（Decoder Output）。这个输出会作为输入传递到下一个解码器模块。整个操作会在接下来的31个解码器模块中重复进行。第32个解码器模块的最终输出会传递到输出层（Output block）。

接下来，让我们在代码中看看这些操作是如何实现的：



In [ ]:
## Step2f: The Decoder Block. The class name is assigned as TransformerBlock to match the name of Meta llama 3 code base.

class TransformerBlock(nn.Module):
  def __init__(self, args: ModelArgs):
    super().__init__()
    self.args = args
    # Initilizate RMSNorm for attention
    self.attention_norm = RMSNorm(dim=args.dim, eps = args.norm_eps)
    # Initilizate Attention class
    self.attention = Attention(args)
    # Initilizate RMSNorm for feedfoward class
    self.ff_norm = RMSNorm(dim=args.dim, eps = args.norm_eps)
    # Initilizate feedfoward class
    self.feedforward = FeedForward(args.dim, 4 * args.dim, args.multiple_of, args.ffn_dim_multiplier)

  def forward(self, x, start_pos, inference):
    # start_pos = token position for inference mode, inference = True for inference and False for training mode
    # i) pass input embedding to attention_norm and then pass to attention block.
    # ii) the output of attention is then added to embedding(before norm)
    h = x + self.attention(self.attention_norm(x), start_pos, inference)

    # i) pass attention output to ff_norm and then pass to the feedforward network.
    # ii) the output of feedforward network is then added to the attention output(before ff_norm)
    out = h + self.feedforward(self.ff_norm(h))
    # Shape: [bsz,seq_len,dim]
    return out


### Test: TransformerBlock ###
# You need take out the triple quotes below to perform testing
"""
x = torch.randn((ModelArgs.max_batch_size, ModelArgs.max_seq_len, ModelArgs.dim), device=device)
transformer_block = TransformerBlock(ModelArgs)
transformer_block_out = transformer_block(x,start_pos=0, inference=False)
print(f"transformer_block_out.shape: {transformer_block_out.shape}")
"""

### Test Results: ###
"""
transformer_block_out.shape: torch.Size([10, 64, 128])
"""

# 第3步：输出模块（Output Block）

最终解码器模块（final decoder block）的输出会被送入输出模块。具体流程如下：

1. 首先，解码器输出会经过RMSNorm归一化。
2. 然后，送入线性层（Linear Layer）以生成logits（未归一化的预测分数）。
3. 接下来，会根据不同的模式执行以下两种操作之一：

   - **推理模式（Inference）**：计算top_p概率，并生成下一个token。当生成的token数量达到最大长度，或下一个token为句子结束符（end of sentence token）时，生成过程停止。
   - **训练模式（Training）**：根据目标标签（target labels）计算损失（loss），并重复训练，直到达到最大训练轮数（max epochs length）。

让我们通过下方的输出模块流程图进一步理解这一过程。

![output_block](images/output_block.webp)
最后，让我们将输入模块、解码器模块和输出模块这三个部分全部整合起来，这样就得到了完整的Llama 3模型。

接下来，让我们实现最终的Llama 3模型代码：

In [ ]:
## Step3: The Output Block
# This is the Llama 3 model. Again, the class name is maintained as Transformer to match with Meta Llama 3 model.

class Transformer(nn.Module):
  def __init__(self, params: ModelArgs):
    super().__init__()
    # set all the ModelArgs in params variable
    self.params = params
    # Initilizate embedding class from the input block
    self.tok_embeddings = nn.Embedding(params.vocab_size, params.dim)

    # Initialize the decoder block and store it inside the ModuleList. 
    # This is because we've 4 decoder blocks in our Llama 3 model. (Official Llama 3 has 32 blocks)
    self.layers = nn.ModuleList()
    for layer_id in range(params.n_layers):
      self.layers.append(TransformerBlock(args=params))

    # Initilizate RMSNorm for the output block
    self.norm = RMSNorm(params.dim, eps = params.norm_eps)

    # Initilizate linear layer at the output block.
    self.output = nn.Linear(params.dim, params.vocab_size, bias=False)

  def forward(self, x, start_pos=0, targets=None):

    # start_pos = token position for inference mode, inference = True for inference and False for training mode
    # x is the batch of token_ids generated from the texts or prompts using tokenizers.
    # x[bsz, seq_len] -> h[bsz, seq_len, dim]
    h = self.tok_embeddings(x)

    # If the target is none, Inference mode is activated and set to "True" and "False" if Training mode is activated.
    if targets is None:
      inference = True
    else:
      inference = False

    # The embeddings (h) will then pass though all the decoder blocks.
    for layer in self.layers:
      h = layer(h, start_pos, inference)

    # The output from the final decoder block will feed into the RMSNorm
    h = self.norm(h)

    # After normalized, the embedding h will then feed into the Linear layer. 
    # The main task of the Linear layer is to generate logits that maps the embeddings with the vocabulary size.
    # h[bsz, seq_len, dim] -> logits[bsz, seq_len, vocab_size]
    logits = self.output(h).float()
    loss = None

    # Inference mode is activated if the targets is not available
    if targets is None:
      loss = None
    # Training mode is activated if the targets are available. And Loss will be calculated for further model training. 
    else:
      loss = F.cross_entropy(logits.view(-1, self.params.vocab_size), targets.view(-1))

    return logits, loss


### Test: Transformer (Llama Model) ###
# You need take out the triple quotes below to perform testing
"""
model = Transformer(ModelArgs).to(ModelArgs.device)
print(model)
"""

我们刚刚搭建好的Llama 3模型看起来非常完美。现在，我们已经可以开始训练流程了。

### 第4步：训练我们的Llama 3模型

训练流程在上一步的输出模块流程图（第3步）中已经给出。如果在开始训练前需要进一步理解，可以再次参考那张流程图。  
接下来，让我们开始编写训练代码。我也会在代码块中提供必要的解释说明。


In [ ]:
## Step 4: Train Llama 3 Model:

# Create a dataset by encoding the entire tiny_shakespeare data token_ids list using the tokenizer's encode function that we've built at the input block section
dataset = torch.tensor(encode(data), dtype=torch.int).to(ModelArgs.device)
print(f"dataset-shape: {dataset.shape}")

# Define function to generate batches from the given dataset
def get_dataset_batch(data, split, args:ModelArgs):
  seq_len = args.max_seq_len
  batch_size = args.max_batch_size
  device = args.device

  train = data[:int(0.8 * len(data))]
  val = data[int(0.8 * len(data)): int(0.9 * len(data))]
  test = data[int(0.9 * len(data)):]

  batch_data = train
  if split == "val":
    batch_data = val

  if split == "test":
    batch_data = test

  # Picking random starting points from the dataset to give random samples for training, validation and testing.

  ix = torch.randint(0, len(batch_data) - seq_len - 3, (batch_size,)).to(device)
  x = torch.stack([torch.cat([token_bos, batch_data[i:i+seq_len-1]]) for i in ix]).long().to(device)
  y = torch.stack([torch.cat([batch_data[i+1:i+seq_len], token_eos]) for i in ix]).long().to(device)

  return x,y

### Test: get_dataset function ###
"""
xs, ys = get_dataset_batch(dataset, split="train", args=ModelArgs)
print([(decode(xs[i].tolist()), decode(ys[i].tolist())) for i in range(len(xs))])
"""

# Define a evaluate loss function to calculate and store training and validation loss for logging and plotting
@torch.no_grad()
def evaluate_loss(model, args:ModelArgs):
  out = {}
  model.eval()

  for split in ["train", "val"]:
    losses = []
    for _ in range(10):      
      xb, yb = get_dataset_batch(dataset, split, args)
      _, loss = model(x=xb, targets=yb)
      losses.append(loss.item())
    out[split] = np.mean(losses)

  model.train()
  return out

# Define a training function to perform model training
def train(model, optimizer, args:ModelArgs):
    epochs = args.epochs
    log_interval = args.log_interval
    device = args.device
    losses = []   
    start_time = time.time()

    for epoch in range(epochs):
        optimizer.zero_grad()

        xs, ys = get_dataset_batch(dataset, 'train', args)
        xs = xs.to(device)
        ys = ys.to(device)
        logits, loss = model(x=xs, targets=ys)
        loss.backward()
        optimizer.step()

        if epoch % log_interval == 0:
            batch_time = time.time() - start_time
            x = evaluate_loss(model, args)
            losses += [x]            
            print(f"Epoch {epoch} | val loss {x['val']:.3f} | Time {batch_time:.3f}")
            start_time = time.time()

    # Print the final validation loss
    print("validation loss: ", losses[-1]['val'])
    # Display the interval losses in plot 
    return pd.DataFrame(losses).plot()

现在，我们已经定义好了训练函数。接下来，使用下面的代码块开始训练，并在训练完成后，通过绘图来观察训练结果。

In [ ]:
## Start training our Llama 3 model
model = Transformer(ModelArgs).to(ModelArgs.device)
optimizer = torch.optim.Adam(model.parameters())

train(model, optimizer, ModelArgs)

![train_trend](images/train_trend.webp)

上图展示了训练损失和验证损失的变化曲线。整个训练过程共进行了2500个epoch。在Google Colab默认GPU和内存配置下，训练大约只用了10分钟，速度非常快。最后一个epoch的验证损失为2.19，考虑到我们的训练数据量和epoch数量，这个结果还算可以。如果要显著降低损失，我们需要增加训练数据量、提升epoch数量，或者采用更高性能的GPU或处理器。

现在，我们已经完成了模型训练。接下来进入最后一步——推理（Inference），看看模型在面对新输入提示时生成文本的表现如何。



### 第5步：Llama 3模型推理

推理流程在输出模块的流程图（第3步）中已经给出。  
让我们开始编写推理代码吧。


In [ ]:
## Step 5: Inference Llama 3 Model:
# This function generates text sequences based on provided prompts using the LLama 3 model we've built and trained.

def generate(model, prompts: str, params: ModelArgs, max_gen_len: int=500, temperature: float = 0.6, top_p: float = 0.9):

    # prompt_tokens: List of user input texts or prompts
    # max_gen_len: Maximum length of the generated text sequence.
    # temperature: Temperature value for controlling randomness in sampling. Defaults to 0.6.
    # top_p: Top-p probability threshold for sampling prob output from the logits. Defaults to 0.9.
    # prompt_tokens = [0]
    bsz = 1  #For inferencing, in general user just input one prompt which we'll take it as 1-batch
    prompt_tokens = token_bos.tolist() + encode(prompts)
    assert len(prompt_tokens) <= params.max_seq_len, "prompt token length should be small than max_seq_len"
    total_len = min(len(prompt_tokens)+max_gen_len, params.max_seq_len)   

    # this tokens matrix is to store the input prompts and all the output that is generated by model.
    # later we'll use the tokenizers decode function to decode this token to view results in text format
    tokens = torch.full((bsz,total_len), fill_value=token_pad.item(), dtype=torch.long, device=params.device)

    # fill in the prompt tokens into the token matrix
    tokens[:,:len(prompt_tokens)] = torch.tensor(prompt_tokens, dtype=torch.long, device=params.device)

    #create a prompt_mask_token for later use to identify if the token is a prompt token or a padding token
    # True if it is a prompt token, False if it is a padding token
    input_text_mask = tokens != token_pad.item()

    #now we can start inferencing using one token at a time from the prompt_tokens list starting with the first position.
    prev_pos = 0
    for cur_pos in range(1, total_len):
      with torch.no_grad():
        logits, _ = model(x=tokens[:,prev_pos:cur_pos], start_pos=prev_pos)
      if temperature > 0:      
        probs = torch.softmax(logits[:, -1]/temperature, dim=-1)
        next_token = sample_top_p(probs, top_p)        
      else:
        next_token = torch.argmax(logits[:, -1], dim=-1)        

      next_token = next_token.reshape(-1)

      # only replace the token if it's a padding token
      next_token = torch.where(input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token)
      tokens[:, cur_pos] = next_token

      prev_pos = cur_pos
      if tokens[:,cur_pos]==token_pad.item() and next_token == token_eos.item():
        break

    output_tokens, output_texts = [], []    

    for i, toks in enumerate(tokens.tolist()):
      # eos_idx = toks.index(token_eos.item())
      if token_eos.item() in toks:
        eos_idx = toks.index(token_eos.item())
        toks = toks[:eos_idx]

      output_tokens.append(toks)
      output_texts.append(decode(toks))
    return output_tokens, output_texts

# Perform top-p (nucleus) sampling on a probability distribution.
# probs (torch.Tensor): Probability distribution tensor derived from the logits.
# p: Probability threshold for top-p sampling.
# According to the paper, Top-p sampling selects the smallest set of tokens whose cumulative probability mass exceeds the threshold p. 
# The distribution is renormalized based on the selected tokens.
def sample_top_p(probs, p):
    probs_sort, prob_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(prob_idx, -1, next_token)    
    # Sampled token indices from the vocabular is returned 
    return next_token

让我们对新的提示进行推理并检查生成的输出

In [ ]:
## Perform the inferencing on user input prompts
prompts = "Consider you what services he has done"
output_tokens, output_texts = generate(model, prompts, ModelArgs)
output_texts = output_texts[0].replace("<|begin_of_text|>", "")
print(output_texts)

## Output ##
"""
Consider you what services he has done o eretrane
adetranytnn i eey i ade hs rcuh i eey,ad hsatsTns rpae,T
eon o i hseflns o i eee ee hs ote i ocal ersl,Bnnlnface
o i hmr a il nwye ademto nt i a ere
h i ees.
Frm oe o etrane o oregae,alh,t orede i oeral
"""


没错，我们可以看到Llama 3模型已经能够对新提示进行推理并生成文本，尽管由于训练数据量和训练轮数有限，输出效果并不是非常理想。我相信，如果有更大规模的训练数据，模型的准确率会有显著提升。

就到这里！我们已经从零开始成功搭建了自己的Llama 3模型，并顺利完成了训练和推理。在Google Colab Notebook上，利用免费GPU和内存，我们在很短时间内实现了文本生成。如果你一路跟着操作下来，我要为你付出的努力表示由衷的祝贺。


### 我的最终感想

Llama 3及其变体目前是LLM领域最受欢迎的开源大模型之一。我认为，能够从零搭建Llama 3，为开发各种有趣的LLM应用打下了坚实的基础。我始终相信知识应当向所有人开放。欢迎你自由使用和修改这些源码，打造属于你自己的个人或专业项目。祝你好运！

---

脑洞建议：鹏哥，如果你想进一步探索，可以尝试将Llama 3与强化学习、知识图谱、多模态融合等前沿技术结合，甚至开发一个能自动自我进化的“AI孵化器”平台，让模型根据实际场景和用户反馈动态成长。未来，或许你的AI能成为下一个开源社区的明星项目！


Thanks a lot for reading!

[Link to Google Colab notebook](https://12ft.io/proxy?q=https%3A%2F%2Fgithub.com%2Ftamangmilan%2Fllama3%2Fblob%2Fmain%2Fbuild_llama3_from_scratch.ipynb)


# 参考

Meta Llama3 Github:https://github.com/meta-llama/llama3


